In [ ]:
!pip install contractions

In [ ]:
import pandas as pd
import string
import re
import nltk
# nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
import numpy as np
from imblearn.over_sampling import BorderlineSMOTE, SMOTE
from imblearn.under_sampling import RandomUnderSampler
import spacy
from tqdm import tqdm
from nltk import word_tokenize
import contractions
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor


In [ ]:
train_file = '../input/commonlitreadabilityprize/train.csv'

In [ ]:
train_data = pd.read_csv(train_file)

In [ ]:
train_lst = train_data["excerpt"].values.tolist()
target_lst = train_data["target"].values

In [ ]:
def remove_URL(text):
	"""
		Remove URLs from a sample string
	"""
	return re.sub(r"https?://\S+|www\.\S+", "", text)

def remove_html(text):
	"""
		Remove the html in sample text
	"""
	html = re.compile(r"<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});")
	return re.sub(html, "", text)

def remove_non_ascii(text):
	"""
		Remove non-ASCII characters 
	"""
	return re.sub(r'[^\x00-\x7f]',r'', text)

def remove_special_characters(text):
	"""
		Remove special special characters, including symbols, emojis, and other graphic characters
	"""
	emoji_pattern = re.compile(
		'['
		u'\U0001F600-\U0001F64F'  # emoticons
		u'\U0001F300-\U0001F5FF'  # symbols & pictographs
		u'\U0001F680-\U0001F6FF'  # transport & map symbols
		u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
		u'\U00002702-\U000027B0'
		u'\U000024C2-\U0001F251'
		']+',
		flags=re.UNICODE)
	return emoji_pattern.sub(r'', text)

def remove_punct(text):
	"""
		Remove the punctuation
	"""
#     return re.sub(r'[]!"$%&\'()*+,./:;=#@?[\\^_`{|}~-]+', "", text)
	return text.translate(str.maketrans('', '', string.punctuation))

In [ ]:
# Clean some punctutations
train_data["excerpt"] = train_data["excerpt"].apply(lambda x: x.lower())
train_data["excerpt"] = train_data["excerpt"].apply(lambda x: contractions.fix(x))
train_data["excerpt"] = train_data["excerpt"].apply(lambda x: remove_URL(x))
train_data["excerpt"] = train_data["excerpt"].apply(lambda x: remove_html(x))
train_data["excerpt"] = train_data["excerpt"].apply(lambda x: remove_non_ascii(x))
train_data["excerpt"] = train_data["excerpt"].apply(lambda x: remove_special_characters(x))
train_data["excerpt"] = train_data["excerpt"].apply(lambda x: remove_html(x))
train_data["excerpt"] = train_data["excerpt"].apply(lambda x: remove_punct(x))

In [ ]:
docs = train_data["excerpt"].values.tolist()

vectorizer = TfidfVectorizer()
bag = vectorizer.fit_transform(docs)

clf = LinearRegression().fit(bag, target_lst)

In [ ]:
test_file = '../input/commonlitreadabilityprize/test.csv'

In [ ]:
test_data = pd.read_csv(test_file)

In [ ]:
test_data["excerpt"] = test_data["excerpt"].apply(lambda x: x.lower())
test_data["excerpt"] = test_data["excerpt"].apply(lambda x: contractions.fix(x))
test_data["excerpt"] = test_data["excerpt"].apply(lambda x: remove_URL(x))
test_data["excerpt"] = test_data["excerpt"].apply(lambda x: remove_html(x))
test_data["excerpt"] = test_data["excerpt"].apply(lambda x: remove_non_ascii(x))
test_data["excerpt"] = test_data["excerpt"].apply(lambda x: remove_special_characters(x))
test_data["excerpt"] = test_data["excerpt"].apply(lambda x: remove_html(x))
test_data["excerpt"] = test_data["excerpt"].apply(lambda x: remove_punct(x))

In [ ]:
docs = test_data["excerpt"].values.tolist()
bag = vectorizer.transform(docs)

pred = clf.predict(bag)

In [ ]:
predictions = pd.DataFrame()
predictions['id'] = test_data['id']
predictions['target'] = pred

In [ ]:
predictions

In [ ]:
predictions.to_csv("submission.csv", index=False)